In [1]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is B8C5-A35A

 Directory of C:\Users\i.glavatskyi\Ironhack\Week_6\lab-spotify-api\code

13/10/2023  01:15    <DIR>          .
13/10/2023  01:15    <DIR>          ..
13/10/2023  01:14    <DIR>          .ipynb_checkpoints
13/10/2023  01:05                99 configspy.py
13/10/2023  01:13             7,121 lap_spotify_API-Copy1.ipynb
13/10/2023  01:15             7,121 lap_spotify_API.ipynb
               3 File(s)         14,341 bytes
               3 Dir(s)  172,938,182,656 bytes free


In [131]:
#from ..private import config_spotipy
import sys
from configspy import *
import spotipy
import json
import pandas as pd
import time

from spotipy.oauth2 import SpotifyClientCredentials


In [132]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [133]:
def extract_song_id(uri):
    # Split the URI by ':'
    parts = uri.split(':')
    # The song ID is the last part
    song_id = parts[-1]
    return song_id


In [140]:
# Function to search for a song and return its URIs (Spotify IDs)
def search_song(title, artist, limit=1):
    song_uris = []  # Initialize a list to store song URIs

    try:
        if artist:
            # Search for the song using both title and artist
            query = f"track:{title} artist:{artist}"
        else:
            # Search for the song using title alone
            query = f"track:{title}"

        # Add a delay before making the API call
        time.sleep(20)

        # Perform the search
        results = sp.search(q=query, type='track', limit=limit)

        # Check if any tracks were found
        if results['tracks']['items']:
            uris = [item['uri'] for item in results['tracks']['items']]
            song_uris.extend(uris)

        # Create a dictionary to store song data
        song_data = {
            'song_name': [],
            'artist': [],
            'spotify_id': []
        }

        for track in results['tracks']['items']:
            song_data['song_name'].append(track['name'])
            song_data['artist'].append(track['artists'][0]['name'])
            song_data['spotify_id'].append(extract_song_id(track['uri']))

        # Create a DataFrame from the song data
        song_df = pd.DataFrame(song_data)

    except Exception as e:
        # Handle exceptions, e.g., if there's an issue with the Spotify API
        print(f"Error: {e}")
        song_df = pd.DataFrame()

    return song_uris, song_df

# Example usage:
title = "Immigrant Song"
artist = "Led Zeppelin"
limit = 1

In [135]:
def get_audio_features(list_of_song_ids):
    audio_features = []
    
    try:
        for song_id in list_of_song_ids:
            # Get audio features for the song
            features = sp.audio_features(tracks=[song_id])
            audio_features.extend(features)
            
    except Exception as e:
        print(f"Error: {e}")
    
    return audio_features

In [136]:
def extract_audio_feature_keys(features):
    # Extract the keys (audio feature names) from the first item in the list
    return list(features[0].keys())


In [137]:
def get_song_info_with_features(title, artist, limit):
    song_info = []
    audio_features_dict = {
        'song_name': "",
        'artist': "",
        'id': "",
        'danceability': 0, 'energy': 0, 'key': 0, 'loudness': 0,
        'mode': 0, 'speechiness': 0, 'acousticness': 0, 'instrumentalness': 0, 'liveness': 0, 
        'valence': 0, 'tempo': 0, 'type': "", 'id': "", 'uri': "", 'track_href': "", 
        'analysis_url': "", 'duration_ms': 0, 'time_signature': 0
    }
    
    song_ids = search_song(title, artist, limit)
    
    if not song_ids:
        return  # Exit the function if no song IDs are found
    display(0)
    # Extract audio feature keys from the first track's audio features
    if song_ids:
        display(1)
        time.sleep(20)
        track_info = get_audio_features(song_ids)
  
    for chunk in [song_ids[i:i+50] for i in range(0, len(song_ids), 50)]:
        audio_features = get_audio_features(chunk)
        
        for features in audio_features:
            # Append the information to the lists in the dictionary
            audio_features_dict['song_name'].append(title)
            audio_features_dict['artist'].append(artist)
            audio_features_dict['id'].append(chunk[i])  # Take the first song ID
            for key in feature_keys:
                audio_features_dict[key].append(features[key])

        # Add a delay before making the next API call
        display(audio_features_dict)
        time.sleep(20)

    return pd.DataFrame(audio_features_dict)


# Example usage:
#song_info_df = get_song_info_with_features(song_data)

In [142]:
title = "Georgia off My Mind"
artist = ""
limit = 5

In [145]:
tracks=get_song_info_with_features(title, artist, limit)
tracks

0

1

Error: expected string or bytes-like object, got 'list'
Error: expected string or bytes-like object, got 'list'


{'song_name': '',
 'artist': '',
 'id': '',
 'danceability': '',
 'energy': '',
 'key': '',
 'loudness': '',
 'mode': '',
 'speechiness': '',
 'acousticness': '',
 'instrumentalness': '',
 'liveness': '',
 'valence': '',
 'tempo': '',
 'type': '',
 'uri': '',
 'track_href': '',
 'analysis_url': '',
 'duration_ms': '',
 'time_signature': ''}

ValueError: If using all scalar values, you must pass an index

In [128]:
#add to the existing dataset
def add_audio_features(df, audio_features_df):
    # Check if both dataframes have the same length
    if len(df) != len(audio_features_df):
        raise ValueError("Dataframes must have the same length.")

    # Concatenate the dataframes horizontally
    extended_df = pd.concat([df, audio_features_df], axis=1)

    return extended_df


In [143]:
song_uris, song_df = search_song(title, artist, limit)
print("Song URIs:", song_uris)
print("Song DataFrame:")
print(song_df)


Song URIs: ['spotify:track:4dLPPSxHgaoy38YmX4yJnV', 'spotify:track:2se33uCsb88PVwbp6ESNGb', 'spotify:track:61QrMlxsE6A0Qyw21RGmbZ', 'spotify:track:4dmyrJKIeq1Y2XYS48SDSA', 'spotify:track:0C8UU7gEofNAoprBkZZih9']
Song DataFrame:
                                           song_name       artist  \
0                                Georgia Off My Mind   Larkin Poe   
1                                Georgia Off My Mind   Larkin Poe   
2                                Georgia off My Mind   Jared Wade   
3  Georgia on My Mind (with Gene Krupa & His Orch...  Anita O'Day   
4                                Georgia Off My Mind   Larkin Poe   

               spotify_id  
0  4dLPPSxHgaoy38YmX4yJnV  
1  2se33uCsb88PVwbp6ESNGb  
2  61QrMlxsE6A0Qyw21RGmbZ  
3  4dmyrJKIeq1Y2XYS48SDSA  
4  0C8UU7gEofNAoprBkZZih9  


In [144]:
time.sleep(20)
audiofeats=get_audio_features(song_uris)
audiofeats

[{'danceability': 0.643,
  'energy': 0.599,
  'key': 2,
  'loudness': -8.541,
  'mode': 1,
  'speechiness': 0.0292,
  'acousticness': 0.0214,
  'instrumentalness': 1.89e-06,
  'liveness': 0.282,
  'valence': 0.591,
  'tempo': 113.982,
  'type': 'audio_features',
  'id': '4dLPPSxHgaoy38YmX4yJnV',
  'uri': 'spotify:track:4dLPPSxHgaoy38YmX4yJnV',
  'track_href': 'https://api.spotify.com/v1/tracks/4dLPPSxHgaoy38YmX4yJnV',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4dLPPSxHgaoy38YmX4yJnV',
  'duration_ms': 234212,
  'time_signature': 4},
 {'danceability': 0.643,
  'energy': 0.599,
  'key': 2,
  'loudness': -8.541,
  'mode': 1,
  'speechiness': 0.0292,
  'acousticness': 0.0214,
  'instrumentalness': 1.89e-06,
  'liveness': 0.282,
  'valence': 0.591,
  'tempo': 113.982,
  'type': 'audio_features',
  'id': '2se33uCsb88PVwbp6ESNGb',
  'uri': 'spotify:track:2se33uCsb88PVwbp6ESNGb',
  'track_href': 'https://api.spotify.com/v1/tracks/2se33uCsb88PVwbp6ESNGb',
  'analysis_url': 'ht

In [91]:
feature_keys = extract_audio_feature_keys(audiofeats)
print(feature_keys)

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']


In [117]:
audio_feature_dict_dummy= {key: None for key in feature_keys}
print(audio_feature_dict_dummy)

{'danceability': None, 'energy': None, 'key': None, 'loudness': None, 'mode': None, 'speechiness': None, 'acousticness': None, 'instrumentalness': None, 'liveness': None, 'valence': None, 'tempo': None, 'type': None, 'id': None, 'uri': None, 'track_href': None, 'analysis_url': None, 'duration_ms': None, 'time_signature': None}


In [77]:
tracks=get_song_info_with_features(title, artist, limit)
tracks

Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


,song_name,artist,id,acousticness,danceability,energy
